# Ref: 
https://analyticsindiamag.com/gluonts-pytorchts-for-time-series-forecasting/

# Plot 

In [ ]:
from fund_price_loader import NAV_DIR
from fund_price_loader import load_dataset

In [ ]:
import os
from gluonts.dataset.util import to_pandas


nav_files = os.listdir(NAV_DIR)
file_path = os.path.join(NAV_DIR, nav_files[800])
print(f"file_path: {file_path}")
dataset = load_dataset(file_path)
to_pandas(list(dataset)[0]).plot(linewidth=2)

In [ ]:
from gluonts.model import prophet
predictor = prophet.ProphetPredictor(
    freq="D", prediction_length=100)
predictions = predictor.predict(dataset)
for entry, forecast in zip(dataset, predictions):
    to_pandas(entry).plot(linewidth=2)
    forecast.plot(color='g', prediction_intervals=[50.0, 90.0])

# Evaluation 

- split Train and Test Dataset 
- calculate RMSE

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import os
from fund_price_loader import NAV_DIR
from gluonts.dataset.util import to_pandas

nav_files = os.listdir(NAV_DIR)
file_path = os.path.join(NAV_DIR, nav_files[800])

In [ ]:
from fund_price_loader import load_split_dataset
from datetime import datetime
train, test = load_split_dataset(file_path, datetime(year=2021, month=5, day=30))

In [ ]:
to_pandas(list(train)[0]).plot(linewidth=2)
to_pandas(list(test)[0]).plot(linewidth=2)

In [ ]:
from evaluator import evaluation
from gluonts.model import prophet

evaluation(prophet.ProphetPredictor, train, test, verbose=True)

# Back Testing Splitting

In [ ]:
!pip install billiard

In [26]:
from datetime import datetime
from datetime import timedelta
from billiard.pool import Pool
from billiard import cpu_count
from functools import partial
from fund_price_loader import split_nav_dataframe_by_end_dates, load_nav_table, load_dataset

def __split_date_generator(start_date, end_date, duration = 7, period = 1):
    """
    A generator generating the dates splitting the nav_table into 
    multiple training and testing for back-testing. 
    
    Args: 
        - start_date: start date of the nav_table. 
        - end_date:   end date of the nav_table. 
        - duration: (number of days) length of the testing dataset. 
        - period:   (number of days) back-testing interval.
    
    Yields: 
        - split_date: end date of the testing data.
        - period_end_date: end date of the training data. 
        
    Note: testing dataset have dates after the training data. 
    """
    period_end_date = start_date + timedelta(days=duration+1)
    while period_end_date <= end_date:
        split_date = period_end_date - timedelta(days=duration)
        yield split_date, period_end_date
        period_end_date = period_end_date + timedelta(days=period)

def __split(x, nav_table=None):
    train_end, test_end = x
    nav_table = load_nav_table(file_path)
    return split_nav_dataframe_by_end_dates(nav_table, train_end, test_end)
def __eval(x, predictor=None):
    train, test = x
    return evaluation(predictor, train, test, verbose=False)

def parallel_run(file_path, predictor, duration, period):
    start_date = nav_table.index.min()
    end_date = nav_table.index.max()
    split_date_gen = split_date_generator(start_date, end_date, duration = duration, period = period)
    train_ends = list(map(lambda x: x[0], split_date_gen))
    with Pool(cpu_count()) as p:
        split_date_gen = split_date_generator(start_date, end_date, duration = duration, period = period)
        train_test_gen = p.imap(partial(
            __split,
            nav_table = None
        ), split_date_gen)
        rmse_gen = p.imap(partial(
            __eval,
            predictor = predictor
        ), train_test_gen)
        rmses = list(rmse_gen)
    return train_ends, rmses

In [27]:
ans = parallel_run(file_path, prophet.ProphetPredictor, 14, 70)

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_s

In [24]:
import pandas as pd
from gluonts.dataset.util import to_pandas
import matplotlib.pylab as plt
performance_table = pd.DataFrame([train_ends, rmses]).T
performance_table.columns = ['date', 'rmse']
performance_table.set_index('date', inplace=True)
dataset = load_dataset(file_path)
performance_table.plot()
to_pandas(list(dataset)[0]).plot(linewidth=2)
plt.show()

NameError: name 'ans' is not defined

In [ ]:
!pip install sharedmem

In [ ]:
nav_table
import sharedmem
def create_share_mem_array(array):
    fp = sharedmem.empty(array.shape, dtype=array.dtype)
    fp[:] = array[:]
    return fp

In [ ]:
fp_value = create_share_mem_array(nav_table.value.values)
fp_date = create_share_mem_array(nav_table.index)
table = pd.DataFrame([fp_date, fp_value]).T

In [ ]:



def train_test_generator(nav_table, split_date_generator):
    for split_date, period_end_date in split_date_generator:
        
        yield train, test
        
split_date_gen = split_date_generator(file_path, duration=28, period = 28 * 5)


In [ ]:
rmse_list = []
dates = []
for train, test, date in split_gen:
    print(date)
    rmse = evaluation(prophet.ProphetPredictor, train, test, verbose=False)
    rmse_list.append(rmse)
    dates.append(date)